In [ ]:
import pandas as pd
# Carregando os datasets
df_2021 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2021.csv", sep=";", encoding="latin1", low_memory=False)
df_2022 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2022.csv", sep=";", encoding="latin1", low_memory=False)
df_2023 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2023.csv", sep=";", encoding="latin1", low_memory=False)
df_2024 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2024.csv", sep=";", encoding="latin1", low_memory=False)
df_2025 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2025.csv", sep=";", encoding="latin1", low_memory=False)

print("Datasets carregados:")
print("2021 ->", df_2021.shape, "linhas e colunas")
print("2022 ->", df_2022.shape, "linhas e colunas")
print("2023 ->", df_2023.shape, "linhas e colunas")
print("2024 ->", df_2024.shape, "linhas e colunas")
print("2025 ->", df_2025.shape, "linhas e colunas")

In [ ]:
# Concatenar em um único dataframe
df_crimes = pd.concat([df_2021, df_2022, df_2023, df_2024, df_2025], ignore_index=True)
print("Dataframe consolidado:", df_crimes.shape)

In [ ]:
print("CHECKPOINT ETAPA 1 - COLETA DE DADOS")
print(f"Total de registros consolidados: {df_crimes.shape[0]:,}")
print(f"Total de colunas: {df_crimes.shape[1]}")
print("Arquivos coletados: dados_2021.csv, dados_2022.csv, dados_2023.csv, dados_2024.csv, dados_2025.csv")


In [ ]:
# Limpeza inicial do dataframe consolidado

# 1. Remover colunas 'unnamed' geradas por excesso de separadores ou células vazias
df_crimes = df_crimes.loc[:, ~df_crimes.columns.str.lower().str.startswith('unnamed')]

# 2. Renomear colunas principais para facilitar o acesso
df_crimes.rename(columns={
    "Sequência": "sequencia",
    "Data Fato": "data_fato", 
    "Hora Fato": "hora_fato",
    "Grupo Fato": "grupo_fato",
    "Tipo Enquadramento": "tipo_enquadramento",
    "Tipo Fato": "tipo_fato",
    "Municipio Fato": "municipio_fato",
    "Local Fato": "local_fato",
    "Bairro": "bairro",
    "Quantidade Vítimas": "quantidade_vitimas",
    "Idade Vítima": "idade_vitima",
    "Sexo Vítima": "sexo_vitma",
    "Cor Vítima": "cor_vitma",
}, inplace=True)

print("Colunas 'Unnamed' removidas e colunas principais renomeadas.")
df_crimes.head()

In [ ]:
# Filtrar o dataframe para conter apenas os registros de Passo Fundo
df_pf = df_crimes[df_crimes['municipio_fato'].str.upper() == 'PASSO FUNDO'].copy() # Usar .copy() para evitar SettingWithCopyWarning
print(f"Shape do DataFrame de Passo Fundo: {df_pf.shape}")
df_pf.head()

In [ ]:
# Verificar as colunas do dataframe de Passo Fundo
print("Colunas do dataframe de Passo Fundo:")
print("=" * 40)
for i, col in enumerate(df_pf.columns, 1):
    print(f"{i:2d}. {col}")
print("=" * 40)
print(f"Total: {len(df_pf.columns)} colunas")

In [ ]:
# limpeza e padronização dos dados

import unicodedata
import numpy as np
import unidecode # Importando a biblioteca unidecode que foi usada em outra célula

# Função para remover acentos (versão unicode)
def remover_acentos(texto):
    if isinstance(texto, str):
        return ''.join(
            c for c in unicodedata.normalize('NFKD', texto) if not unicodedata.combining(c)
        )
    return texto

# 1. Remover duplicatas
df_pf = df_pf.drop_duplicates()

# 2. Substituir "Sem informação" por NaN em todo o DataFrame
df_pf.replace("Sem informação", np.nan, inplace=True)

# 3. Identificar colunas numéricas e categóricas
numericas = df_pf.select_dtypes(include=['int64', 'float64']).columns
categoricas = df_pf.select_dtypes(include=['object']).columns

# 4. Preencher NaN em numéricas com a média
for col in numericas:
    df_pf[col] = df_pf[col].fillna(df_pf[col].mean())

# 5. Preencher NaN em categóricas com "ignorado" e padronizar o texto
for col in categoricas:
    df_pf[col] = df_pf[col].fillna("ignorado")
    # Padronizar: minúsculas, remover espaços e acentos
    df_pf[col] = (
        df_pf[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .apply(remover_acentos) # Usando a função de remoção de acentos
    )

# 6. Converter colunas de data e hora para datetime
df_pf['data_fato'] = pd.to_datetime(df_pf['data_fato'], format='%d/%m/%Y', errors='coerce')
df_pf['hora_fato'] = pd.to_datetime(df_pf['hora_fato'], format='%H:%M:%S', errors='coerce').dt.time

# 7. Garantir que colunas originalmente numéricas permaneçam numéricas
for col in numericas:
    df_pf[col] = pd.to_numeric(df_pf[col], errors='coerce')

# Conferir o resultado
print("DataFrame tratado e consolidado:")
print(f"Novo shape após limpeza: {df_pf.shape}")
df_pf.head()

In [ ]:
# Seleciona novamente as colunas categóricas após o tratamento
categoricas_verificacao = df_pf.select_dtypes(include=['object']).columns

for col in categoricas_verificacao:
    print(f"\nValores únicos em {col}:")
    # Limita a exibição para não poluir a saída
    unique_values = df_pf[col].unique()
    if len(unique_values) > 15:
        print(f" (Exibindo os primeiros 15 de {len(unique_values)})")
        print(unique_values[:15])
    else:
        print(unique_values)

Todas as linhas duplicadas foram removidas usando `drop_duplicates()`.

Qualquer ocorrência de "Sem informação" foi substituída por `NaN` para uniformizar os valores ausentes.

**Colunas numéricas** (ex.: `idade_vitima`, `quantidade_vitimas`): valores ausentes foram preenchidos com a média da coluna, garantindo que não haja lacunas para cálculos estatísticos ou agregações.

**Colunas categóricas** (ex.: `sexo_vitma`, `cor_vitma`, `tipo_fato`): valores ausentes foram preenchidos com a categoria "ignorado", preservando a consistência e permitindo análises categóricas sem perder linhas.

Todas as colunas categóricas foram padronizadas:
- Convertidas para minúsculas.
- Espaços extras no início/fim foram removidos.
- Acentos foram eliminados (ex: `Homicídio` → `homicidio`).

Coluna `data_fato` convertida para datetime no formato `YYYY-MM-DD`.

Coluna `hora_fato` convertida para `datetime.time`, permitindo manipulação temporal correta.

Todas as colunas originalmente numéricas foram mantidas como `int64` ou `float64`, evitando conversão involuntária para string durante a padronização.

O DataFrame agora está:
- Livre de duplicatas.
- Sem valores ausentes ou inconsistentes.
- Com dados categóricos padronizados e uniformes.
- Com datas e horas em formatos apropriados.
- Pronto para integração com outros datasets ou sistemas de análise.